In [1]:
import numpy as np
import pandas as pd
import copy
import math
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,StratifiedKFold
from collections import deque
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
from torch.autograd import Variable

# What the data looks like

3 columns:
1. Equation 1
2. Equation 2
3. Label

Need to split the data by character and then join cols 1 and 2 with each other, with a seperator in between.

Need to create a character dictionary that maps values to integars

In [2]:
df = pd.read_csv('/Users/morgan/Downloads/balancedTrainingSet.csv')

In [3]:
df.head(5)

,(0i1),(0i1).1,1
0,(0i1),(0i1&0i2),1
1,(0i1),(0i1&1i0),1
2,(0i1),(0i1&1i2),1
3,(0i1),(0i1&2i0),1
4,(0i1),(0i1&2i1),1


In [6]:
N = df.shape[0]
print(N)

1999999


# Shape the csv values

Add divider

In [7]:
divider = np.full(N,'@')

In [8]:
df['divider'] = divider

In [9]:
df['input'] = df['(0i1)'] + df['divider'] + df['(0i1).1']

In [10]:
df['input'].head(5)

0    (0i1)@(0i1&0i2)
1    (0i1)@(0i1&1i0)
2    (0i1)@(0i1&1i2)
3    (0i1)@(0i1&2i0)
4    (0i1)@(0i1&2i1)
Name: input, dtype: object

In [11]:
df.head(5)

,(0i1),(0i1).1,1,divider,input
0,(0i1),(0i1&0i2),1,@,(0i1)@(0i1&0i2)
1,(0i1),(0i1&1i0),1,@,(0i1)@(0i1&1i0)
2,(0i1),(0i1&1i2),1,@,(0i1)@(0i1&1i2)
3,(0i1),(0i1&2i0),1,@,(0i1)@(0i1&2i0)
4,(0i1),(0i1&2i1),1,@,(0i1)@(0i1&2i1)


#### Split the input and expand into new df

In [30]:
inputs = df['input']

In [31]:
inputs.head()

0    (0i1)@(0i1&0i2)
1    (0i1)@(0i1&1i0)
2    (0i1)@(0i1&1i2)
3    (0i1)@(0i1&2i0)
4    (0i1)@(0i1&2i1)
Name: input, dtype: object

In [71]:
inputs = inputs.str.rstrip()

In [75]:
inputs = inputs.apply(lambda x: pd.Series(list(x)))

In [29]:
# inputs = df.iloc[:,4].str.split().str.get(0).str.split('',expand=True)

#### Clean resulting df

Replace '' and None with zero padding.

In [10]:
# inputs.drop([0], axis=1,inplace=True)
# inputs.drop([30], axis=1,inplace=True)

In [78]:
print(inputs.shape)
print(inputs.head(5))

(1999999, 39)
  0  1  2  3  4  5  6  7  8  9   ...   29   30   31   32   33   34   35   36  \
0  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

    37   38  
0  NaN  NaN  
1  NaN  NaN  
2  NaN  NaN  
3  NaN  NaN  
4  NaN  NaN  

[5 rows x 39 columns]


#### Turn None into empty string

In [79]:
# mask = inputs.applymap(lambda x: x is None)
# cols = inputs.columns[(mask).any()]
# for col in inputs[cols]:
#     inputs.loc[mask[col], col] = ''

In [80]:
# print(inputs.head(5))

  0  1  2  3  4  5  6  7  8  9   ...   29   30   31   32   33   34   35   36  \
0  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4  (  0  i  1  )  @  (  0  i  1  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

    37   38  
0  NaN  NaN  
1  NaN  NaN  
2  NaN  NaN  
3  NaN  NaN  
4  NaN  NaN  

[5 rows x 39 columns]


#### Turn empty string into character denoting white space

In [81]:
mask = inputs.applymap(lambda x: x is np.nan)
cols = inputs.columns[(mask).any()]
for col in inputs[cols]:
    inputs.loc[mask[col], col] = '<pad>'

In [82]:
# print(inputs.iloc[0,13])
# print(type(inputs[0]))
print(inputs.head(5))
print(type(inputs))
print(inputs.shape)
print(inputs.columns)

  0  1  2  3  4  5  6  7  8  9   ...     29     30     31     32     33  \
0  (  0  i  1  )  @  (  0  i  1  ...  <pad>  <pad>  <pad>  <pad>  <pad>   
1  (  0  i  1  )  @  (  0  i  1  ...  <pad>  <pad>  <pad>  <pad>  <pad>   
2  (  0  i  1  )  @  (  0  i  1  ...  <pad>  <pad>  <pad>  <pad>  <pad>   
3  (  0  i  1  )  @  (  0  i  1  ...  <pad>  <pad>  <pad>  <pad>  <pad>   
4  (  0  i  1  )  @  (  0  i  1  ...  <pad>  <pad>  <pad>  <pad>  <pad>   

      34     35     36     37     38  
0  <pad>  <pad>  <pad>  <pad>  <pad>  
1  <pad>  <pad>  <pad>  <pad>  <pad>  
2  <pad>  <pad>  <pad>  <pad>  <pad>  
3  <pad>  <pad>  <pad>  <pad>  <pad>  
4  <pad>  <pad>  <pad>  <pad>  <pad>  

[5 rows x 39 columns]
<class 'pandas.core.frame.DataFrame'>
(1999999, 39)
RangeIndex(start=0, stop=39, step=1)


In [83]:
labels = df.iloc[:,2].values

# Save modified data

In [90]:
inputs.to_csv('cleanedRaleighData.csv')

# ML Data

### For transformer

_ will be the end of line character.


### For MLP

Turn the dataset into 1 hot encoded vectors, with zero padding for end of line.

# Create a dictionary of labels

break up the inputs into single characters

In [84]:
from collections import defaultdict
d = defaultdict(preprocessing.LabelEncoder)
le = preprocessing.LabelEncoder()

## Get all unique occurances in the dataset to feed into the encoder

In [85]:
def return_uniques(values):
    uniques, count = np.unique(values, return_counts=True)
    print(uniques, count)
    return uniques, count

In [86]:
uniques,counts = return_uniques(inputs)

['&' '(' ')' '0' '1' '2' '<pad>' '@' 'i' 'x' '|'] [ 5267649  6944727  6944727  8234252  8151150  8039350 17261002  1999999
  6596280  5616096  2944729]


In [87]:
le.fit(uniques)

LabelEncoder()

In [88]:
num_classes = len(list(le.classes_))
print('num_classes',num_classes)
list(le.classes_)

num_classes 11


['&', '(', ')', '0', '1', '2', '<pad>', '@', 'i', 'x', '|']

In [89]:
return_uniques(labels)

[0 1] [1000000  999999]


(array([0, 1]), array([1000000,  999999]))

# Transform the inputs to categorical

In [91]:
ml_inputs = inputs.apply(lambda x: le.transform(x))

In [92]:
np_inputs = ml_inputs.values

In [93]:
np_inputs[:5]

array([[1, 3, 8, 4, 2, 7, 1, 3, 8, 4, 0, 3, 8, 5, 2, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       [1, 3, 8, 4, 2, 7, 1, 3, 8, 4, 0, 4, 8, 3, 2, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       [1, 3, 8, 4, 2, 7, 1, 3, 8, 4, 0, 4, 8, 5, 2, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       [1, 3, 8, 4, 2, 7, 1, 3, 8, 4, 0, 5, 8, 3, 2, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
       [1, 3, 8, 4, 2, 7, 1, 3, 8, 4, 0, 5, 8, 4, 2, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]])

In [94]:
labels[:5]

array([1, 1, 1, 1, 1])

# Create the data sets

Because the data is heavily mismatched, We should try a limited but equal dataset. And after that try training more frequently on the 1 labels

In [95]:
X = np_inputs
y = labels
print(X.shape,y.shape)

(1999999, 39) (1999999,)


In [418]:
# mask_1s = y == 1
# mask_0s = y == 0

In [419]:
# X1 = X[mask_1s]
# y1 = y[mask_1s]

# X0 = X[mask_0s]
# y0 = y[mask_0s]

In [424]:
# indexes = np.arange(X0.shape[0])
# restricted_mask = np.random.choice(indexes,X1.shape[0],replace=False)

In [430]:
# restrictedX0 = X0[restricted_mask]
# restrictedy0 = y0[restricted_mask]

In [431]:
# print(X1.shape)
# print(X0.shape)
# print(restrictedX0.shape)

(5473, 29)
(994526, 29)
(5473, 29)


In [432]:
# restrictedX = np.concatenate([restrictedX0,X1],axis=0)
# restrictedy = np.concatenate([restrictedy0,y1],axis=0)

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [97]:
class mathDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, X,y):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        sample = {'item': X[idx], 'label': y[idx]}

        return sample

In [98]:
trainset = mathDataset(X_train,y_train)
valset = mathDataset(X_val,y_val)
testset = mathDataset(X_test,y_test)

trainloader = DataLoader(trainset, batch_size=512,
                        shuffle=True, num_workers=4)
valloader = DataLoader(valset, batch_size=512,
                        shuffle=False, num_workers=4)
testloader = DataLoader(testset, batch_size=512,
                        shuffle=False, num_workers=4)

In [99]:
data = {
    'trainset':trainloader,
    'valset':valloader,
    'testset':testloader
}

# Transformer

We only need a binary classification in the output. So in a very real sense we don't even need a decoder, just a feed forward binary classification NN?

In [135]:
class Embedder(nn.Module):
    def __init__(self,vocab_size,d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size,d_model)
    def forward(self,x):
        return self.embed(x)

class PositionalEncoder(nn.Module):
    def __init__(self,d_model, maxlen_seq = 39):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(maxlen_seq,d_model)
        for pos in range(maxlen_seq):
            for i in range(0,d_model,2):
                pe[pos,i] = pos / math.sin(10000 ** ((2*i)/d_model))
                pe[pos,i+1] = pos / math.cos(10000 ** ((2*(i+1))/d_model))
        pe.unsqueeze(0)
        self.register_buffer('pe',pe)
        
    def forward(self,x):
        x = x * math.sqrt(self.d_model)
        seq_len = x.size(-1)
        x = x + Variable(self.pe[:,:self.d_model],requires_grad=False)
        return x
    
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output
    
def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    if mask is not None:
        mask = mask.unsqueeze(1).unsqueeze(3)
        # print('scores,mask',scores.size(),mask.size())
        scores = scores.masked_fill(mask == 0, -1e9)
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
    # print('scores, v',scores.size(), v.size())
    output = torch.matmul(scores, v)
    return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x
    
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x
# We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = torch.concat([hour,day,month,target])
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)
    
class ClassificationOut(nn.Module):
    def __init__(self,d_model,N):
        super().__init__()
        self.N = N
        self.layers = get_clones(FeedForward(d_model),N)
        
    def forward(self,x):
        for i in range(self.N):
            x = self.layers[i](x)
        return x
        
class Transformer(nn.Module):
    def __init__(self, src_vocab,N_categories,maxlen, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = ClassificationOut(d_model,N)
        self.out = nn.Linear(maxlen*d_model, N_categories)
    def forward(self, src, src_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(e_outputs)
        flat_d = d_output.view(d_output.size(0),-1)
        output = self.out(flat_d)
        return output
# we don't perform softmax on the output as this will be handled 
# automatically by our loss function

# Masks

I don't need any masks aside from the input padding attention mask, because the output is not a series.

# Test the transformer

In [118]:
d_model = 256
N = 2
N_categories = 2
heads = 8
vocab_size = num_classes
maxlen = 39

test_net = Transformer(vocab_size,N_categories,maxlen, d_model, N, heads)

In [119]:
test_data = np_inputs[:5]#.reshape(1,29)
print(test_data.shape,test_data)
torch_test = torch.from_numpy(test_data)
src_msk = torch_test != 6

(5, 39) [[1 3 8 4 2 7 1 3 8 4 0 3 8 5 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6]
 [1 3 8 4 2 7 1 3 8 4 0 4 8 3 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6]
 [1 3 8 4 2 7 1 3 8 4 0 4 8 5 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6]
 [1 3 8 4 2 7 1 3 8 4 0 5 8 3 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6]
 [1 3 8 4 2 7 1 3 8 4 0 5 8 4 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6]]


In [120]:
# a = torch.ones((5,maxlen,256))
# b = a.view(a.size(0),-1)

In [121]:
# print(b.size())
print(src_msk.size())
print(torch_test.size())

torch.Size([5, 39])
torch.Size([5, 39])


In [122]:
# a = torch.ones((5,8,maxlen,maxlen))
# b = torch.zeros((5,1,maxlen,1))
# a = a.masked_fill(b == 0, -1e9)

In [123]:
preds = test_net(torch_test,src_msk)

scores,mask torch.Size([5, 8, 39, 39]) torch.Size([5, 1, 39, 1])
scores, v torch.Size([5, 8, 39, 39]) torch.Size([5, 8, 39, 32])
scores,mask torch.Size([5, 8, 39, 39]) torch.Size([5, 1, 39, 1])
scores, v torch.Size([5, 8, 39, 39]) torch.Size([5, 8, 39, 32])


In [124]:
print(preds.shape)

torch.Size([5, 2])


In [125]:
preds

tensor([[-0.0281,  0.0152],
        [-0.0116,  0.0257],
        [-0.0206,  0.0086],
        [-0.0329,  0.0161],
        [-0.0099, -0.0063]], grad_fn=<AddmmBackward>)

In [126]:
F.softmax(preds,dim=1)

tensor([[0.4892, 0.5108],
        [0.4907, 0.5093],
        [0.4927, 0.5073],
        [0.4878, 0.5122],
        [0.4991, 0.5009]], grad_fn=<SoftmaxBackward>)

In [128]:
F.cross_entropy(preds.view(1,10),torch.tensor([0]))

tensor(2.3265, grad_fn=<NllLossBackward>)

# Train the transformer

In [136]:
def train_model(model,data_dict,train_params):
    start_time = time.time()
    scores_window = deque(maxlen=100)
    scores = []
    for e in range(train_params['epochs']):
        for i, data in enumerate(data_dict['trainset'], 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data.values()
            src_mask = inputs != 6
            preds = model(inputs,src_mask)
#             print('preds',preds,'y',labels)
            optim.zero_grad()
            loss = train_params['criterion'](preds,labels)
            loss.backward()
            optim.step()
            scores_window.append(loss.data)
            scores.append(loss.data)
            if (i + 1) % train_params['print_every'] == 0:
                loss_avg = np.mean(scores_window)
                print("time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % 
                      ((time.time() - start_time) // 60,e + 1, i + 1, loss_avg, time.time() - start_time,
                train_params['print_every']))
            if i == train_params['episodes']:
                break
    return model,scores
    

# Add Hyperparam search and LR stepper

In [137]:
d_model = 512
N = 2
N_categories = 2
heads = 8
maxlen = 39
vocab_size = num_classes

model = Transformer(vocab_size,N_categories,maxlen, d_model, N, heads)

criterion = F.cross_entropy
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

train_params = {
    'epochs':10,
    'episodes':50,
    'optimizer':optim,
    'criterion': criterion,
    'print_every':10
}

In [138]:
trained_model,scores = train_model(model,data,train_params)

time = 2m, epoch 1, iter = 10, loss = 0.627,166s per 10 iters
time = 5m, epoch 1, iter = 20, loss = 0.535,340s per 10 iters


KeyboardInterrupt: 

# Test the model

In [440]:
for i in range(3):
    d = next(iter(data['trainset']))
    inputs,label = d.values()
    print(inputs.size(),label.size())
    src_mask = inputs != 6
    preds = trained_model(inputs,src_mask)
    print(F.softmax(preds,dim=1))
    nonzero_mask = label == 1
    print(label[nonzero_mask],F.softmax(preds,dim=1)[nonzero_mask])
#     print(labels - F.softmax(preds,dim=1))
    

torch.Size([512, 29]) torch.Size([512])
tensor([[9.9679e-01, 3.2067e-03],
        [9.9441e-01, 5.5931e-03],
        [9.5624e-01, 4.3756e-02],
        ...,
        [9.9461e-01, 5.3859e-03],
        [9.9883e-01, 1.1724e-03],
        [9.9958e-01, 4.2087e-04]], grad_fn=<SoftmaxBackward>)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) tensor([[0.9349, 0.0651],
        [0.7700, 0.2300],
        [0.4588, 0.5412],
        [0.8108, 0.1892],
        [0.9732, 0.0268],
        [0.4862, 0.5138],
        [0.4204, 0.5796],
        [0.8902, 0.1098],
        [0.8161, 0.1839],
        [0.8129, 0.1871],
        [0.7815, 0.2185],
        [0.9193, 0.0807],
        [0.1583, 0.8417],
        [0.7052, 0.2948],
        [0.4678, 0.5322],
        [0.9621, 0.0379],
        [0.4987, 0.5013]], grad_fn=<IndexBackward>)
torch.Size([512, 29]) torch.Size([512])
tensor([[9.8417e-01, 1.5834e-02],
        [2.8159e-01, 7.1841e-01],
        [9.7386e-01, 2.6136e-02],
        ...,
        [9.9993e-01, 7.4016e-05]

# NN for predicting labels

In [141]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [333]:

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

def hard_update(source,target):
    for target_param,param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(param.data)
        
class Critic(nn.Module):
    def __init__(self,seed,nS,nA,hidden_dims=(128,64,64,32)):
        super(Critic,self).__init__()
        self.seed = torch.manual_seed(seed)
        self.nS = nS
        self.nA = nA
        
        self.emb = nn.Embedding(11,40)
        self.input_layer = nn.Linear(40,hidden_dims[0])
        self.input_bn = nn.BatchNorm1d(hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.Linear(hidden_dims[0],hidden_dims[1]))
        for i in range(1,len(hidden_dims)-1):
            hidden_layer = nn.Linear(hidden_dims[i],hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        # self.fc1 = nn.Linear(hidden_dims[0]+nA,hidden_dims[1])
        # self.fc1_bn = nn.BatchNorm1d(hidden_dims[1])
        self.output_layer = nn.Linear(hidden_dims[-1],nA)
        self.reset_parameters()
        
    def reset_parameters(self):
        self.input_layer.weight.data.uniform_(*hidden_init(self.input_layer))
        for hidden_layer in self.hidden_layers:
            hidden_layer.weight.data.uniform_(*hidden_init(hidden_layer))
        self.output_layer.weight.data.uniform_(-3e-3,3e-3)
        
    def forward(self,state):
        # With batchnorm
        # xs = self.input_bn(F.relu(self.input_layer(state)))
        # x = torch.cat((xs,action),dim=1)
        # x = self.fc1_bn(F.relu(self.fc1(x)))
        embs = self.emb(state)#.view((1, -1))
        x = F.relu(self.input_layer(embs))
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        x = self.output_layer(x)
        return F.log_softmax(x,dim=0)

# Instantiate the NN.

predict 2 classes

In [252]:
X = np_inputs
y = labels
print(X.shape,vanilla_y.shape)

In [193]:
nS = X.shape[1]
nA = 2
seed = 7

In [334]:
net = Critic(seed,nS,nA)
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Split the data into train,val,test

1 hot encode the labels

In [254]:
one_hot = preprocessing.OneHotEncoder(categories='auto')
one_hot.fit(labels.reshape(-1,1))

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [255]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [256]:
data = [X_train,y_train, X_val, y_val, X_test, y_test]
[print(d.shape) for d in data]

(639999, 29)
(639999,)
(160000, 29)
(160000,)
(200000, 29)
(200000,)


[None, None, None, None, None, None]

In [257]:
# Convert to torch tensor
torch_data = []
for d in data:
    torch_data.append(torch.from_numpy(d).long())

In [258]:
torch_X_train,torch_y_train, torch_X_val, torch_y_val, torch_X_test, torch_y_test = torch_data

# Train NN

In [271]:
episodes = 10
for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data.values()
    one_hot_labels = one_hot.transform(labels.reshape(-1,1)).toarray()
    if i == 0:
        break;

In [275]:
print(inputs.shape,inputs)
print(one_hot_labels.shape,one_hot_labels)

torch.Size([2048, 29]) tensor([[1, 3, 8,  ..., 7, 7, 7],
        [1, 3, 8,  ..., 9, 3, 2],
        [1, 3, 8,  ..., 7, 7, 7],
        ...,
        [1, 4, 9,  ..., 7, 7, 7],
        [1, 3, 8,  ..., 7, 7, 7],
        [1, 3, 8,  ..., 8, 3, 2]])
(2048, 2) [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [335]:
for e in range(1,episodes+1):
#     score_window = deque(maxlen=100)
    # running_loss = 0
    scores_window = deque(maxlen=100)
    # for i, data in enumerate(trainloader, 0):
    #     # get the inputs; data is a list of [inputs, labels]
    #     inputs, labels = data.values()
    #     one_hot_labels = one_hot.transform(labels.reshape(-1,1)).toarray()
		# y = torch.from_numpy(one_hot_labels).long()
		# X = torch.from_numpy(X).long()
    # print(inputs)
    # print(labels)
    # print(type(inputs[0][0]))
    # print(type(inputs),type(labels))
    # print('torch_hot_labels',torch_hot_labels)
    # print(inputs.size(),torch_hot_labels.size())
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, torch_hot_labels)
    loss.backward()
    optimizer.step()

    # print statistics
    scores_window.append(loss.item())
    # running_loss += loss.item()
    # if i == 0:
    #     break;
    print(np.mean(scores_window))
        # if i % 25 == 0:    # print every 25 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (e, i, running_loss / 25))
        #     running_loss = 0.0

7.624871730804443
7.624739170074463
7.624658584594727
7.624699592590332
7.624795436859131
7.624780178070068
7.624799728393555
7.624707221984863
7.624665260314941
7.624787330627441


In [345]:
import random
names = ['marmoset,tamarin,capuchin,lemur,crocodile,alligator,hippopotamus']
N = 5
N = len(names) if N > len(names) else N
stripped_routes = [
    
]
historical_routes = np.zeros(5)
historical_routes = np.vstack((stripped_routes,historical_routes))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [348]:
random.shuffle(names)
print(names)
random.shuffle(names)
print(names)

['marmoset,tamarin,capuchin,lemur,crocodile,alligator,hippopotamus']
['marmoset,tamarin,capuchin,lemur,crocodile,alligator,hippopotamus']


# Try different architectures

- Vanilla NN
- Transformer
- LSTM
- XGBoost

# XGBoost with PBT and multiprocessing

In [322]:
from multiprocessing import Process,Pool,RawArray
import xgboost as xgb
from xgboost import XGBClassifier,XGBRegressor, plot_importance
import time

In [330]:
class PBTV2(object):
    def __init__(self,boost_params,attribute_dict):
        """
        boost_params : dictionary of default xgboost params
        attribute_dict : dictionary of hyperparameters for PBT + dictionaries for random/scaled param search
        
        mu: Total population
        tau: offspring per step. Must be less than mu (typically 1/6ish)
        
        Sample x from the top performers
        Sample x from the highest diversity of the population
        
        modifications:
        mutation of one or more params
        combination of two parent genes
        
        Random sample from initial distribution
        Scale by a factor between 0.8 and 1.2 
        
        Scores: dict, round -> scores of all agents.
        
        Individuals: np.array - shape(mu,len(params.keys) + scores + diversity)
        param_dict: dictionary of param keys for generating random values within a range
        param_constraints: dictionary of param keys : tuple([low,high],dtype) for scaling params within a range
        set_dict: dictionary of param keys and values that are unchanging
        """
        self.mu = attribute_dict['mu']
        self.tau = attribute_dict['tau']
        self.seed = attribute_dict['seed']
        self.N_islands = attribute_dict['islands']
        self.metrics = attribute_dict['metrics']
        self.model_type = attribute_dict['model_type']
        self.population_dict = {}
        self.scores = []
        self.combined_metric = []
        self.historical_params = []
        self.params = boost_params
        self.useable_mask = np.ones(len(self.params.keys()),dtype=bool)
        mask = np.array([-4,-3,-2,-1])
        self.useable_mask[mask] = 0
        self.param_dict = attribute_dict['param_dict']
        # For scaling params (some params don't scale)
        self.param_constraints = attribute_dict['param_constraints']
        self.param_index = {k:i for i,k in enumerate(self.param_dict.keys())}
        self.set_dict = attribute_dict['set_dict']
        self.set_keys = ['num_round','n_estimators','scale_pos_weight','objective']
        self.set_values = [30,1000,1,'reg:squarederror']
        self.diversity_discount = np.array([0.199,4,7,7,0.5,0.5,0.5,0.5])
        self.diversity_subtract = np.array([0.001,1,3,3,0.5,0.5,0,0])
        self.N_params = len(self.param_dict.keys())
        self.N_cols = self.N_params + 3 #+ score and diversity and combined
        self.score_col = self.N_params
        self.diversity_col = self.N_params + 1
        self.combined_col = self.N_params + 2
        self.M_shape = (self.mu,self.N_cols)
        for island in range(self.N_islands):
            self.population_dict[island] = self.instantiate_params()
            # Now add score and diversity
    #         self.linear_return_fitness(self.individuals)
            self.population_dict[island] = self.return_fitness(self.population_dict[island])
            self.population_dict[island] = self.return_diversity(self.population_dict[island])
            self.population_dict[island] = self.return_combined_score(self.population_dict[island])
            # Record for prosperity
            self.scores.append({0:copy.copy(self.population_dict[island][:,self.score_col])})
            self.combined_metric.append({0:copy.copy(self.population_dict[island][:,self.combined_col])})
            self.historical_params.append({0:copy.copy(self.population_dict[island])})
        
    def np_return_model(self,params):
        learning_rate,max_depth,min_child_weight,gamma,subsample,colsample_bytree,L2,L1 = params
        model = self.model_type(num_round = self.set_dict['num_round'],
        n_estimators=self.set_dict['n_estimators'],
        objective=self.set_dict['objective'],
        scale_pos_weight=self.set_dict['scale_pos_weight'],
        learning_rate = learning_rate,
        max_depth=int(max_depth),
        min_child_weight=int(min_child_weight),
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_lambda=L2,
        reg_alpha=L1,
        seed=self.seed)
        return model
        
    def instantiate_params(self):
        population = np.zeros((self.mu,self.N_cols))
        for i,key in enumerate(self.param_dict.keys()):
            column = [self.param_dict[key]() for row in range(self.mu)]
            population[:,i] = np.asarray(column)
        return population
    
    def fitness(self,params):
#         print('params',params)
        model = self.np_return_model(params)
        model = PBTV2.XGBoostTrain(self.X_train, self.X_valid, self.y_train, self.y_valid, model)
        y_pred = model.predict(self.X_valid)
        loss = np.mean(abs(self.y_valid - y_pred))
        print('Mean Absolute Error: ',loss)
        return loss
    
    def linear_return_fitness(self,group):
        params = group[:,:self.N_params]
        losses = []
        for row in params:
            loss = self.fitness(row)
            losses.append(loss)
            
    def return_fitness(self,population):
        N = population.shape[0]
        params = population[:,:self.N_params]
        param_list = list(params)
        print('param_list',param_list)
        with Pool(processes=4) as pool:
            results = pool.map(outside_fitness,param_list)
            print('results',results)
        population[:,self.score_col] = np.array(results).reshape(1,N)
        return population
        
    def return_diversity(self,population):
        """
        Each category must be weighted appropriately
        'learning_rate' : '',
        'min_child_weight': /100,
        'max_depth': /100,
        'gamma': /100,
        'subsample': '',
        'colsample_bytree': '',
        'reg_lambda': '',
        'reg_alpha': ''
        The rest are summed up as is
        """
        median_params = np.median(population[:,:self.N_params],axis=0)
        difference = population[:,:self.N_params] - median_params.reshape(1,self.N_params)
        difference -= self.diversity_subtract
        difference /= self.diversity_discount
        diversity_scores = np.sum(difference,axis=1)
        population[:,self.diversity_col] = np.abs(diversity_scores)
        return population
    
    def return_combined_score(self,population):
        """
        combine diversity and score to get a combined metric
        """
        combined = population[:,self.score_col] - population[:,self.diversity_col]
        population[:,self.combined_col] = combined
        return population
            
    def train(self,num_rounds):
        for i in range(1,num_rounds+1):
            self.step(i)
    
    def step(self,index):
        """
        could sample with probability of priority as given by the score
        
        Take the tau top performing candidates. There should also a term that values uniqueness in terms of params.
        """
        print('Step: ',index)
        for island in range(0,self.N_islands):
            scores = self.population_dict[island][:,self.metrics[island]]
            scores = np.sort(scores)
            cutoff = scores[-self.tau]
            elite_cutoff = scores[self.tau]
            print(scores,cutoff)
            print('first score',np.min(scores))
            print('last score',np.max(scores))
            diversity = self.population_dict[island][:,self.diversity_col]
            diversity = np.sort(diversity)
            replaced_mask = np.where(self.population_dict[island][:,self.metrics[island]] >= cutoff)[0]
            elite_mask = np.where(self.population_dict[island][:,self.metrics[island]] < cutoff)[0] # can use elite cutoff to restrict who propogates into the next generation
            for idx in replaced_mask:
                self.population_dict[island][idx,:] = self.recombine(self.population_dict[island],self.population_dict[island][idx,:],elite_mask)
    #             func = choice([self.mutate,self.scale_param])
                self.population_dict[island][idx,:] = self.mutate(self.population_dict[island][idx,:])
                self.population_dict[island][idx,:] = self.mutate(self.population_dict[island][idx,:])
#                 self.population_dict[island][idx,:] = self.scale_param(self.population_dict[island][idx,:])
            self.population_dict[island][replaced_mask] = self.return_fitness(self.population_dict[island][replaced_mask])
            self.population_dict[island][replaced_mask] = self.return_diversity(self.population_dict[island][replaced_mask])
            self.population_dict[island][replaced_mask] = self.return_combined_score(self.population_dict[island][replaced_mask])
            
            self.scores[island][index] = copy.copy(self.population_dict[island][:,self.score_col])
            self.combined_metric[island][index] = copy.copy(self.population_dict[island][:,self.combined_col])
            self.historical_params[island][index] = copy.copy(self.population_dict[island])
        
    ### Mutation Methods ###
    
    def mutate(self,target):
        key = choice(list(self.param_dict.keys()))
        value = self.param_dict[key]()
        index = self.param_index[key]
        target[index] = value
        return target
        
    def scale_param(self,target):
        """
        scale value to search locally. Must be constrained between the min and max values however.
        """
        key = choice(list(self.param_dict.keys()))
        index = self.param_index[key]
        value = np.random.random() * 0.4 + 0.8 # value between 0.8 - 1.2
        target[index] *= value
        constraint = self.param_constraints[key][0]
        low = constraint[0]
        high = constraint[1]
        dtype = self.param_constraints[key][1]
        target[index] = dtype(min(max(target[index],low),high))
        return target
    
    @staticmethod
    def random_parents(population,elite_mask):
        """Get two random parents from a population.
        :return (Individual, Individual): Two random parents.
        """
        subgroup = population[elite_mask]
        split = choice(range(1, subgroup.shape[0] - 1))
        return choice(subgroup[:split]), choice(subgroup[split:])
        
        
    def recombine(self,population,target,elite_mask):
        parent_one,parent_two = PBTV2.random_parents(population,elite_mask)
        # randomly selects a gene from one
        np.concatenate([parent_one[:self.N_params],parent_two[:self.N_params]])
        for index in range(self.N_params):
            gene_one,gene_two = parent_one[index],parent_two[index]
            gene = np.random.choice([gene_one,gene_two])
            target[index] = gene
        # Splits the genes and copy
#         divide = np.random.choice(list(range(self.N_params))) # for splitting the two.
#         genes_one = parent_one[:divide]
#         genes_two = parent_two[divide:self.N_params]
#         target[:self.N_params] = np.concatenate([genes_one,genes_two])
        return target

In [331]:
def XGBoostTrain(X_train, X_valid, y_train, y_valid,model):
    eval_set = [(X_train, y_train), (X_valid, y_valid)]
    eval_metric = ["rmse"]
    model.fit(X_train, y_train,eval_metric=eval_metric, eval_set=eval_set, verbose=False)
    print(model)
    # return_accuracy(model,X_valid,y_valid)
    return model

def outside_fitness(params,load=True):
    model_type = XGBClassifier
    model = outside_return_model(params,model_type)
    # Load data from trainer if applicable.
    if load == True:
        losses = []
        trainset = data['trainset']
        valset = data['valset']
        # Sample N times from trainloader
        N = len(trainset)
        V = len(valset)
        N_index = np.random.choice(N,50)
        V_index = np.random.choice(V,10)
        inputs, labels = trainset[N_index].values()
        V_inputs, V_labels = valset[V_index].values()
        model = XGBoostTrain(inputs, V_inputs, labels, V_labels, model)
        y_pred = model.predict(V_labels)
        batch_loss = np.mean(abs(V_labels - y_pred))
        loss = np.mean(losses)
    else:
        model = XGBoostTrain(data['X_train'], data['X_valid'], data['y_train'], data['y_valid'], model)
        y_pred = model.predict(data['X_valid'])
        loss = np.mean(abs(data['y_valid'] - y_pred))
    print('Mean Absolute Error: ',loss)
    return loss

def outside_return_model(params,model_type):
    learning_rate,max_depth,min_child_weight,gamma,subsample,colsample_bytree,L2,L1 = params
    model = model_type(num_round = 30,
    n_estimators=1000,
    objective='reg:squarederror',
    scale_pos_weight=1,
    learning_rate = learning_rate,
    max_depth=int(max_depth),
    min_child_weight=int(min_child_weight),
    gamma=gamma,
    subsample=subsample,
    colsample_bytree=colsample_bytree,
    reg_lambda=L2,
    reg_alpha=L1,
    seed=27)
    return model

In [332]:
mu = 5
tau = 2
seed = 28
islands = 1
N_params = 8
target_index = 0 # 0 = spread, 1 = total
attribute_dict = {'mu':mu,
                 'tau':tau,
                 'seed':seed,
                 'metrics':[N_params,N_params + 2],
                  'model_type':XGBClassifier,
                  'islands':islands,
                 'param_dict' :{
                'learning_rate' : lambda: np.random.sample() * 0.199 + 0.001,
                'min_child_weight': lambda:int(np.random.sample() * 4 + 1),
                'max_depth': lambda:int(np.random.sample() * 7 + 3),
                'gamma': lambda:np.random.sample() * 7 + 3,
                'subsample': lambda:np.random.sample() * 0.5 + 0.5,
                'colsample_bytree': lambda:np.random.sample() * 0.5 + 0.5,
                'reg_lambda': lambda:np.random.sample() * 0.5,
                'reg_alpha': lambda:np.random.sample() * 0.5,
                },
                # For scaling params (some params don't scale)
                'param_constraints' :{
                'learning_rate' : ([0,0.5],float),
                'min_child_weight': ([0,5],int),
                'max_depth': ([3,10],int),
                'gamma': ([3,10],float),
                'subsample': ([0.5,1],float),
                'colsample_bytree': ([0.5,1],float),
                'reg_lambda': ([0,0.5],float),
                'reg_alpha': ([0,0.5],float),
                },
                'set_dict':{'num_round':30,'n_estimators':1000,'scale_pos_weight':1,'objective':'reg:squarederror'}}
params = {
     'learning_rate' :0.01,
     'max_depth':10,
     'min_child_weight':1,
     'gamma':0,
     'subsample':0.8,
     'colsample_bytree':0.8,
     'scale_pos_weight':1,
      'reg_lambda':1,
      'reg_alpha':0,
     'objective':'reg:squarederror',
    'num_round': 30,
     'n_estimators':1000}

data = {
    'trainset':trainset,
    'valset':valset,
    'testset':testset
}
# data = {
#     'X_train':X_train,
#     'X_valid':X_valid,
#     'y_train':y_train,
#     'y_valid':y_valid,
#     'X':X
# }

populationV2 = PBTV2(params,attribute_dict)

param_list [array([0.15088505, 3.        , 9.        , 7.0934881 , 0.97707955,
       0.79813886, 0.10524555, 0.07279346]), array([0.0410704 , 1.        , 5.        , 6.70459189, 0.9586591 ,
       0.75919257, 0.21721193, 0.09741529]), array([0.09798658, 4.        , 4.        , 7.26248433, 0.61444035,
       0.95284021, 0.16355721, 0.33061735]), array([0.02369729, 2.        , 5.        , 6.9139313 , 0.92573661,
       0.59141349, 0.24894362, 0.23613034]), array([0.12769532, 2.        , 5.        , 3.71736133, 0.61411493,
       0.63430885, 0.01787642, 0.25270027])]
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.7591925738185055,
       gamma=6.704591893961149, learning_rate=0.041070404460732644,
       max_delta_step=0, max_depth=1, min_child_weight=5, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None, num_round=30,
       objective='reg:squarederror', random_state=0,
       reg_alpha=0.09741529230153484, 

ValueError: Input numpy.ndarray must be 2 dimensional

# T-SNE

In [ ]:
def color_scatter(X,colors):
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts